In [33]:
import numpy as np
import math

# Massas molares g/mol
h2 = 2.016 
co = 28.01 
co2 = 44.01 
n2 = 28.0134 
ch3oh = 32.04
h2o = 18.01528

NE = 1 # número da equipe
cco = (53.5 + (NE * 0.25)) / 100 # conversão CO
exh2 = (12.8 + NE * 0.2) # excesso H2

def molarmassa(C, el): # converte fração massica para molar 
    return(C / el)

 

In [34]:
class Corrente: 
    pass

c = {} # correntes
for i in range(1, 24 + 1):
    c[i] = Corrente()
    
ch3ohC22 = 10000 # kg / h
c[22].ch3oh = molarmassa(ch3ohC22, ch3oh)
c[22].vm = c[22].ch3oh  / 0.999 # vm = vazão molar
c[22].h2o  = c[22].vm * 0.001

Reação Principal: CO + 2 H<sub>2</sub> &rarr; CH<sub>3</sub>OH <br>
Reação Secundária: CO<sub>2</sub> + 3 H<sub>2</sub> &rarr; CH<sub>3</sub>OH + H<sub>2</sub>O

# Abaixo Balanço sem reciclo

In [35]:
# BM sem reciclo
c[1].vm = 1000 # kmol vm = vazão molar
c[1].fco2 = 0.04 # f fração molar
c[1].fn2 = 0.01

while True: # trocar por um while true e um if break na condição
    for c[1].fco in np.arange( 0, 0.95 + 0.00001, 0.00001): # variando a proporção h2 + co da entrada  
        c[1].fh2 = 0.95 - c[1].fco # restrição, reajustar como limite de loop talvez
        # c6 reação 1
        c[1].ch3oh = (c[1].fco * cco * c[1].vm)
        cor1 = (1 - cco) * (c[1].fco * c[1].vm)
        h2r1 = (c[1].fco * cco * c[1].vm * 2) # consumido h2
        # c6 reação 2
        ch3ohr2 = (c[1].fco2 * c[1].vm)
        h2or2 = (c[1].fco2 * c[1].vm)
        h2r2 = 3 * (c[1].fco2 * c[1].vm)
        # c6
        c[6].h2 = c[1].vm * c[1].fh2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
        c[6].ch3oh = c[1].ch3oh + ch3ohr2
        c[6].co = (c[1].fco * c[1].vm) - cor1
        c[6].n2 = c[1].fn2 * c[1].vm
        c[6].h2o = h2or2
        c[6].vm = c[6].h2 + c[6].ch3oh + c[6].co + c[6].n2 + h2or2
        # excesso h2 na reação 1
        eh2 = (c[1].fh2 * c[1].vm) - h2r1 
        Eh2 = exh2 * h2r1
        if math.isclose(eh2, Eh2, rel_tol = 1e-3): # restrição excesso
            break
    # VS-1
    # c13
    c[13].ch3oh = 0.98 *  c[6].ch3oh
    c[13].h2o =  0.96 * c[6].h2o
    c[13].fh2 = 0.001000
    c[13].fco = 0.000044
    c[13].fn2 = 0.000058
    c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
    c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
    c[13].n2 = c[13].vm * c[13].fn2
    c[13].h2 = c[13].vm * c[13].fh2
    c[13].co = c[13].vm * c[13].fco

    # c9 
    c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
    c[9].h2o =  c[6].h2o - c[13].h2o
    c[9].vm = c[6].vm - c[13].vm
    c[9].n2 = c[6].n2 - (c[13].fn2 * c[13].vm)
    c[9].h2 = c[6].h2 - c[13].h2
    c[9].co = c[6].co - c[13].co
    c[9].fh2 = c[9].h2 / c[9].vm
    c[9].fco = c[9].co / c[9].vm
    c[9].fn2 = c[9].n2 / c[9].vm
    c[9].fch3oh = c[9].ch3oh / c[9].vm
    c[9].fh2o = c[9].h2o / c[9].vm
    # VS-2
    # c16
    c[16].ch3oh = 0.995 * c[13].ch3oh
    c[16].h2o =  0.98 * c[13].h2o
    c[16].fh2 = 0.000950
    c[16].fco = 0.000040
    c[16].fn2 = 0.000053
    c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
    c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
    c[16].n2 = c[16].vm * c[16].fn2
    c[16].co = c[16].vm * c[16].fco
    c[16].h2 = c[16].vm * c[16].fh2

    # c15
    c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
    c[9].h2o =  c[13].h2o - c[16].h2o
    c[15].vm = c[13].vm - c[16].vm
    c[15].n2 = c[13].n2 - (c[13].fn2 * c[16].vm)

    # DE-1 
    # c21
    c[21].vm = c[22].vm

    # c20
    c[20].vm = c[21].vm + c[22].vm
    c[20].ch3oh = c[22].ch3oh * 2
    c[20].h2o = c[22].h2o * 2 

    # c20
    c[17].n2 = c[16].n2
    c[17].co = c[16].co
    c[17].h2 = c[16].h2
    c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
    c[17].ch3oh = c[20].ch3oh

    # c19
    c[19].n2 = c[17].n2
    c[19].co = c[17].co
    c[19].h2 = c[17].h2
    c[19].vm = c[19].n2 + c[19].co + c[19].h2

    # c24
    c[24].fh2o = 0.94
    c[24].fch3oh = 1 - c[24].fh2o
    c[24].h2o = c[16].h2o - c[22].h2o
    c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
    c[24].vm = c[24].ch3oh + c[24].h2o

    if c[24].vm < 0 or c[24].ch3oh < 0: # caso contrario 
        c[1].vm = c[1].vm + 100

    elif math.isclose(c[16].vm, c[17].vm + c[24].vm, rel_tol=1e-3):
        break
    elif c[17].vm + c[24].vm - c[16].vm > 0:
        c[1].vm = c[1].vm + 1
    elif c[17].vm + c[24].vm - c[16].vm < 0:
        c[1].vm = c[1].vm - 1

print(c[1].vm)


9000


In [36]:
# BM com reciclo
# R = c[9].vm / c[11].vm

while True: # trocar por um while true e um if break na condição
    for R in np.arange(0.0001, 1 + 0.0001, 0.0001):
        for c[1].fco in np.arange( 0, 0.95 + 0.00001, 0.00001): # variando a proporção h2 + co da entrada      
            # c11
            c[11].vm = c[9].vm / R 
            c[11].fh2 = c[9].fh2
            c[11].fco = c[9].fco
            c[11].fn2 = c[9].fn2
            c[11].fch3oh = c[9].fch3oh
            c[11].fh2o = c[9].fh2o
            # c1
            c[1].fh2 = 0.95 - c[1].fco # restrição, reajustar como limite de loop talvez
            # c3
            c[3].vm = c[1].vm + c[11].vm
            c[3]
            # c6 reação 1 AJUSTAR PARA C3
            c[3].ch3oh = (c[3].fco * cco * c[3].vm)
            cor1 = (1 - cco) * (c[3].fco * c[3].vm)
            h2r1 = (c[3].fco * cco * c[3].vm * 2) # consumido h2
            # c6 reação 2
            ch3ohr2 = (c[3].fco2 * c[3].vm)
            h2or2 = (c[3].fco2 * c[3].vm)
            h2r2 = 3 * (c[3].fco2 * c[3].vm)
            # c6
            h2c6 = c[1].vm * c[1].fh2 - (h2r1 + h2r2) # h2 em excesso que vai para c6
            c[6].ch3oh = c[1].ch3oh + ch3ohr2
            c[6].co = (c[1].fco * c[1].vm) - cor1
            c[6].n2 = c[1].fn2 * c[1].vm
            c[6].h2o = h2or2
            c[6].vm = h2c6 + c[6].ch3oh + c[6].co + c[6].n2 + h2or2
            # excesso h2 na reação 1
            eh2 = (c[1].fh2 * c[1].vm) - h2r1 
            Eh2 = exh2 * h2r1
            if math.isclose(eh2, Eh2, rel_tol = 1e-3): # restrição excesso
                break
        # VS-1
        # c13
        c[13].ch3oh = 0.98 *  c[6].ch3oh
        c[13].h2o =  0.96 * c[6].h2o
        c[13].fh2 = 0.001000
        c[13].fco = 0.000044
        c[13].fn2 = 0.000058
        c[13].fch3oh_h2o = 1 - (c[13].fco + c[13].fh2 + c[13].fn2) # fração combinada ch3oh e h2o
        c[13].vm = (c[13].ch3oh + c[13].h2o) / c[13].fch3oh_h2o
        c[13].n2 = c[13].vm * c[13].fn2

        # c9 
        c[9].ch3oh =  c[6].ch3oh - c[13].ch3oh
        c[9].h2o =  c[6].h2o - c[13].h2o
        c[9].vm = c[6].vm - c[13].vm
        c[9].n2 = c[6].n2 - (c[13].fn2 * c[13].vm)
        
        # R

        # VS-2
        # c16
        c[16].ch3oh = 0.995 * c[13].ch3oh
        c[16].h2o =  0.98 * c[13].h2o
        c[16].fh2 = 0.000950
        c[16].fco = 0.000040
        c[16].fn2 = 0.000053
        c[16].fch3oh_h2o = 1 - (c[16].fco + c[16].fh2 + c[16].fn2) # fração combinada ch3oh e h2o
        c[16].vm = (c[16].ch3oh + c[16].h2o) / c[16].fch3oh_h2o
        c[16].n2 = c[16].vm * c[16].fn2
        c[16].co = c[16].vm * c[16].fco
        c[16].h2 = c[16].vm * c[16].fh2

        # c15
        c[15].ch3oh = c[13].ch3oh - c[16].ch3oh
        c[9].h2o =  c[13].h2o - c[16].h2o
        c[15].vm = c[13].vm - c[16].vm
        c[15].n2 = c[13].n2 - (c[13].fn2 * c[16].vm)

        # DE-1 
        # c21
        c[21].vm = c[22].vm

        # c20
        c[20].vm = c[21].vm + c[22].vm
        c[20].ch3oh = c[22].ch3oh * 2
        c[20].h2o = c[22].h2o * 2 

        # c20
        c[17].n2 = c[16].n2
        c[17].co = c[16].co
        c[17].h2 = c[16].h2
        c[17].vm = c[17].n2 + c[17].co + c[17].h2 + c[20].vm
        c[17].ch3oh = c[20].ch3oh

        # c19
        c[19].n2 = c[17].n2
        c[19].co = c[17].co
        c[19].h2 = c[17].h2
        c[19].vm = c[19].n2 + c[19].co + c[19].h2

        # c24
        c[24].fh2o = 0.94
        c[24].fch3oh = 1 - c[24].fh2o
        c[24].h2o = c[16].h2o - c[22].h2o
        c[24].ch3oh = c[16].ch3oh - c[17].ch3oh
        c[24].vm = c[24].ch3oh + c[24].h2o

    if c[24].vm < 0 or c[24].ch3oh < 0: # caso contrario 
        c[1].vm = c[1].vm + 100

    elif math.isclose(c[16].vm, c[17].vm + c[24].vm, rel_tol=1e-3):
        break
    elif c[17].vm + c[24].vm - c[16].vm > 0:
        c[1].vm = c[1].vm + 1
    elif c[17].vm + c[24].vm - c[16].vm < 0:
        c[1].vm = c[1].vm - 1

print(c[1].vm)

AttributeError: 'Corrente' object has no attribute 'fco'